In [48]:
import pandas as pd

class AcsData:
    def __init__(self,filepath="data/acs.csv"):
        self.filepath = filepath
        self.df = self.load_data()
        self.rename_columns()

    def create_summary(self):
        races = ['white', 'black', 'native', 'pacific_islander', 'other', 'asian','hispanic']

        summary = self.df.groupby('fips').sum()
        for race in races:
            col_name = race + '_percentage'
            summary[col_name] = summary[race] / summary['total_population']

        summary.reindex(columns=['fips'])
        return summary

    def rename_columns(self):
        fips = self.df['GEO.id2'].astype('str').str[:5]

        hispanic_values = ['HD01_VD12','HD01_VD13','HD01_VD14','HD01_VD15','HD01_VD16','HD01_VD17','HD01_VD18',
                           'HD01_VD19','HD01_VD20','HD01_VD21']

        race_cols = ['HD01_VD01','HD01_VD03','HD01_VD04','HD01_VD05','HD01_VD07','HD01_VD08','HD01_VD06']

        combined_cols = hispanic_values + race_cols
        df = self.df[combined_cols]
        df[combined_cols] = df[combined_cols].astype(int)
        df['fips'] = fips

        df['hispanic'] = df[hispanic_values].sum(axis=1)
        df.drop(hispanic_values,axis=1,inplace=True)
        col_names = {
            'HD01_VD01': 'total_population',
            'HD01_VD03': 'white',
            'HD01_VD04': 'black',
            'HD01_VD05': 'native',
            'HD01_VD07': 'pacific_islander',
            'HD01_VD08': 'other',
            'HD01_VD06': 'asian',
            'fips': 'fips',
            'hispanic':'hispanic'
        }

        df.rename(columns=col_names, inplace=True)
        self.df = df

    def load_data(self):
        community_data = pd.read_csv(self.filepath, encoding="ISO-8859-1")
        data = community_data[1:]
        return data

In [117]:
import pandas as pd

class Stop:
    def __init__(self,filepath):
        self.df = self.load_dataframe(filepath)

    def load_dataframe(self, filepath):
        df = pd.read_csv(filepath)
        cols_to_drop = ['location_raw', 'county_name','driver_race_raw']
        df = df.drop(cols_to_drop)
        df['driver_race'] = df['driver_race'].str.lower()
        df['county_fips'] = df['county_fips'].astype(str)
        return df

    def create_summary(self):
        summary = self.df.group

In [118]:
filepath = 'data/stop_data/WY-clean.csv'
stops = Stops(filepath)

/Users/shawnveltman/anaconda3/envs/geo/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2961: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [119]:
df = stops.df
df.county_fips.unique()

array([56025., 56039., 56037., 56031., 56035., 56021., 56029., 56023.,
       56041., 56015., 56027., 56009., 56003., 56019., 56011., 56007.,
       56045., 56017., 56001., 56005.,    nan, 56043., 56013., 56033.])

In [167]:
df['driver_race'] = df['driver_race'].str.lower()
summary = df.groupby(['county_fips','driver_race']).agg('count')
cols = summary.columns
summary = summary[['id']]
summary['individual'] = summary['id']
summary = summary[['individual']]
summary['percentage'] = summary['individual'] / summary['individual'].groupby(level=0).sum()
summary

individual  percentage
county_fips driver_race                        
56001.0     asian               312    0.017473
            black               499    0.027946
            hispanic           1178    0.065972
            other                42    0.002352
            white             15825    0.886257
56003.0     asian                37    0.014324
            black                13    0.005033
            hispanic             93    0.036005
            other                20    0.007743
            white              2420    0.936895
56005.0     asian                57    0.010294
            black                66    0.011920
            hispanic            278    0.050208
            other                20    0.003612
            white              5116    0.923966
56007.0     asian               403    0.023850
            black               550    0.032550
            hispanic           1166    0.069006
            other                63    0.003728
            white             14715    0.870865
56009.0     asian                90    0.015045
            black                98    0.016382
            hispanic            377    0.063022
            other                39    0.006520
            white              5378    0.899030
56011.0     asian                72    0.018311
            black                84    0.021363
            hispanic            129    0.032808
            other                19    0.004832
            white              3628    0.922686
...                             ...         ...
56035.0     asian               156    0.016449
            black                75    0.007908
            hispanic            540    0.056938
            other                25    0.002636
            white              8688    0.916069
56037.0     asian               263    0.018720
            black               351    0.024984
            hispanic           1055    0.075094
            other               119    0.008470
            white             12261    0.872731
56039.0     asian               151    0.024257
            black                56    0.008996
            hispanic            449    0.072129
            other                38    0.006104
            white              5531    0.888514
56041.0     asian               132    0.014354
            black               236    0.025663
            hispanic            604    0.065681
            other                45    0.004893
            white              8179    0.889408
56043.0     asian                32    0.012408
            black                26    0.010081
            hispanic            100    0.038775
            other                20    0.007755
            white              2401    0.930981
56045.0     asian                12    0.007895
            black                18    0.011842
            hispanic             58    0.038158
            other                 3    0.001974
            white              1429    0.940132

[115 rows x 2 columns]

In [141]:
summary[56001.0]

driver_race
asian         312
black         499
hispanic     1178
other          42
white       15825
dtype: object